# welcome to celldb!

Functional genomics data is often very sparse, difficult to normalize ahead of time, and sometimes very large.

celldb is here to help make that easier by offering an extensible data model over industry standard technologies. Namely, the Phoenix SQL adapter let's us write SQL to HBase, which means bioinformaticians can easily write map-reduce jobs!

So we'll start by running a docker instance that is pre-loaded with a Phoenix-Hbase stack compatible with celldb.

Run this command in a separate terminal:

```sudo docker run -p 8765:8765 -it david4096/docker-phoenix```.

## Initializing tables

The first time celldb starts we initialize the databases used for storing our functional genomics data, which are organized into samples and features. This only needs to be done once.

In [1]:
import celldb
connection = celldb.connect("http://localhost:8765")
celldb.initialize(connection)

(u'Table already exists. tableName=EXPRESSIONS', 1013, u'42M04', None)
(u'Table already exists. tableName=FEATURES', 1013, u'42M04', None)


Note that we were returned a cursor. We'll return to cursors in a second. But if you still have the terminal available for the docker-phoenix container you should see messages something like this:

```
17/06/30 20:49:11 INFO client.HBaseAdmin: Created SYSTEM.MUTEX
17/06/30 20:49:11 INFO metrics.Metrics: Initializing metrics system: phoenix
17/06/30 20:49:12 INFO impl.MetricsConfig: loaded properties from hadoop-metrics2.properties
17/06/30 20:49:12 INFO impl.MetricsSystemImpl: Scheduled snapshot period at 10 second(s).
17/06/30 20:49:12 INFO impl.MetricsSystemImpl: phoenix metrics system started
17/06/30 20:49:16 INFO client.HBaseAdmin: Created SYSTEM.CATALOG
17/06/30 20:49:24 INFO client.HBaseAdmin: Created SYSTEM.SEQUENCE
17/06/30 20:49:25 INFO client.HBaseAdmin: Created SYSTEM.STATS
17/06/30 20:49:27 INFO client.HBaseAdmin: Created SYSTEM.FUNCTION
17/06/30 20:49:28 INFO client.HBaseAdmin: Created EXPRESSIONS
17/06/30 20:49:29 INFO client.HBaseAdmin: Created FEATURES
```

This activity log shows we've created tables the celldb needs to represent functional genomics data.

## Cursors

The celldb python client is DB2.0 compliant, which means you can easily integrate it into your existing python application.

To interact with the database, the programmer uses a cursor which can execute queries and coalesce results.

For example, to list the (empty) `Expressions` table we might write:

In [4]:
cursor = connection.cursor()
cursor.execute("SELECT * from Expressions")
cursor.fetchall()

[[u'sample_A']]

The celldb client offers a number of convenience functions so that one can interact with the data without having to write SQL.

## Upserting samples

RNA expression quantifications tell us, for a given sample, just how much a given gene was expressed. We will design a simple sample here, called `sample_A` and we will quantify two genes, `gene_1` and `gene_2` with values `0.5` and `1.0` respectively.

Upserting simply means we are either inserting or updating, and since samples and identified uniquely by their sampleId, a single value can be updated without inserting a new column or updated the entire row.

In [6]:
sampleId = "sample_A"
features = ["gene_1", "gene_2"]
values = [0.5, 1.0]
cursor = celldb.upsert_sample(cursor, sampleId, features, values)

## Listing samples

A convenience method is offered for collecting the list of sampleIds.

In [6]:
samples = celldb.list_samples(cursor)
print(samples)

[u'sample_A']


## Listing features

We can also discover the features that have been quantified:

In [7]:
features = celldb.list_features(cursor)
print(features)

[u'gene_1', u'gene_2']


## Retrieving a matrix

RNA expression analysis centers on gene-cell matrices, and so a function is provided which simplifies gathering these data.

In [9]:
celldb.matrix(cursor, samples, features)

[[u'sample_A', Decimal('0.5'), Decimal('1')]]

If you have pandas, you can easily load these data into a dataframe using the matrix_sql method. Note the usage of all caps index column.

In [17]:
import pandas as pd
sql = celldb.matrix_sql(samples, features)
df = pd.read_sql(sql, connection, index_col="SAMPLEID")
df

,GENE_1,GENE_2
SAMPLEID,,
sample_A,0.5,1.0
